In [ ]:
from VC_collections import Collection
from VC_collections.Collection import connect_to_google_drive

import gspread

In [ ]:
cred = Collection.get_google_drive_credentials()

In [ ]:
client = gspread.authorize(cred)

In [ ]:
len(client.list_spreadsheet_files())

In [1]:
from VC_collections.AuthorityFiles import Authority_instance

In [ ]:
Authority_instance.df_privacy_values

In [2]:
Authority_instance.privacy_search_dict

{'אין מגבלת פרטיות': 'אין מגבלת פרטיות',
 'ללא מגבלות פרטיות': 'אין מגבלת פרטיות',
 'בלי מגבלת פרטיות': 'אין מגבלת פרטיות',
 'ללא': 'אין מגבלת פרטיות',
 'לא': 'אין מגבלת פרטיות',
 'פרטיות נתונים אישים': 'פרטיות נתונים אישיים',
 'פרטיות - נתונים אישים': 'פרטיות נתונים אישיים',
 'פרטיות מידע רגיש': 'פרטיות מידע רגיש',
 'פרטיות - מידע רגיש': 'פרטיות מידע רגיש',
 'פרטיות מידע רפואי': 'פרטיות מידע רפואי',
 'פרטיות - מידע רפואי': 'פרטיות מידע רפואי',
 'פרטיות מידע סוציאלי (רווחה)': 'פרטיות מידע סוציאלי (רווחה)',
 'פרטיות - מידע סוציאלי (רווחה)': 'פרטיות מידע סוציאלי (רווחה)',
 'פרטיות תנאי העסקה': 'פרטיות תנאי העסקה',
 'פרטיות - תנאי העסקה': 'פרטיות תנאי העסקה',
 'פרטיות קטינים': 'פרטיות קטינים',
 'פרטיות - קטינים': 'פרטיות קטינים',
 'פרטיות אחר': 'פרטיות אחר',
 'פרטיות - אחר': 'פרטיות אחר',
 'פרטיות לבדיקה': 'פרטיות לבדיקה',
 'פרטיות - לבדיקה': 'פרטיות לבדיקה',
 'לבירור': 'פרטיות לבדיקה',
 'פרטיות הסכמים חוזים': 'פרטיות הסכמים חוזים',
 'פרטיות - הסכמים חוזים': 'פרטיות הסכמים חוזים'}

In [ ]:
import pandas as pd

In [ ]:
search_dict = pd.Series(Authority_instance.df_privacy_values["רמיזות"].apply(lambda x: x.split(";")), 
                                                               index = Authority_instance.df_privacy_values.index.values).to_dict()

In [ ]:
new_search_dict = {}

for key, value in search_dict.items():
    if len(value)>1:
        for item in value:
            new_search_dict[item] = key
    else:
        new_search_dict[value[0]] = key

In [ ]:
new_search_dict